### 과제 url

url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

 Challenge는 Optional Task로 필수적으로 해야하는 것은 아닙니다. 좀 더 심화된 크롤링을 원할경우 시도해보면 좋을 것 같아요!

In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.15.2


In [18]:
# ['category', 'title', 'codimap_date', 'views', 'comment_numbers']

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

codi_list = []
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

interval = 1
prev_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    # Wait to load page
    time.sleep(interval)
    
    # Calculate new scroll height and compare with last scroll height
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break
    
    prev_height = curr_height
    

columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
values = []
soup = BeautifulSoup(driver.page_source, 'lxml')
data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

for i, row in enumerate(data_rows):
    
    print('{}번째 데이터를 수집합니다'.format(i+1))
    blank = []
    
    category = row.find('span', attrs={'class':'style-list-information__text'})
    if category:
        category = category.get_text().strip()
        blank.append(category)
    else:
        blank.append('Something is wrong')
        print('{}번째 카테고리 가져올 때 문제발생'.format(i+1))
        continue
    
    
    title = row.find('strong', attrs={'class':'style-list-information__title'})
    if title:
        title = title.get_text().strip()
        blank.append(title)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디 이름 가져올 때 문제발생'.format(i+1))
        continue
    
    
    date = row.find_all('span', attrs={'class':'post-information__text'})[0]
    if date:
        date = date.get_text().strip()
        blank.append(date)
    else:
        blank.append('Something is wrong')
        print('{}번째 날짜 가져올 때 문제발생'.format(i+1))
        continue
    
    
    views = row.find_all('span', attrs={'class':'post-information__text'})[1]
    if views:
        views = views.get_text().strip()
        blank.append(views)
    else:
        blank.append('Something is wrong')
        print('{}번째 조회수 가져올 때 문제발생'.format(i+1))
        continue
    
    
    comment = row.find_all('span', attrs={'class':'post-information__text'})[2]
    if comment:
        comment = comment.get_text().strip()
        blank.append(comment)
    else:
        blank.append('Something is wrong')
        print('{}번째 댓글수 가져올 때 문제발생'.format(i+1))
        continue
    
    
    values.append(blank)
    print('------------------------------------')
    
df = pd.DataFrame(values, columns=columns)
codi_list.append(df)    



1번째 데이터를 수집합니다
------------------------------------
2번째 데이터를 수집합니다
------------------------------------
3번째 데이터를 수집합니다
------------------------------------
4번째 데이터를 수집합니다
------------------------------------
5번째 데이터를 수집합니다
------------------------------------
6번째 데이터를 수집합니다
------------------------------------
7번째 데이터를 수집합니다
------------------------------------
8번째 데이터를 수집합니다
------------------------------------
9번째 데이터를 수집합니다
------------------------------------
10번째 데이터를 수집합니다
------------------------------------
11번째 데이터를 수집합니다
------------------------------------
12번째 데이터를 수집합니다
------------------------------------
13번째 데이터를 수집합니다
------------------------------------
14번째 데이터를 수집합니다
------------------------------------
15번째 데이터를 수집합니다
------------------------------------
16번째 데이터를 수집합니다
------------------------------------
17번째 데이터를 수집합니다
------------------------------------
18번째 데이터를 수집합니다
------------------------------------
19번째 데이터를 수집합니다
------------------------------------
20

In [19]:
df_ = pd.concat(codi_list).reset_index(drop=True)
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"조회 17,315",댓글 50
1,캐주얼,블랙 매니아,21.02.23,"조회 36,354",댓글 47
2,캐주얼,오늘의 코디,22.03.22,"조회 13,424",댓글 44
3,댄디,추천 댄디 룩,21.09.15,"조회 21,619",댓글 32
4,캐주얼,릴랙스 룩,21.02.23,"조회 24,050",댓글 25
5,스트릿,힙하다 힙해,21.02.03,"조회 36,107",댓글 24
6,댄디,데이트 코디,21.12.24,"조회 10,994",댓글 19
7,캐주얼,심플한 코디,21.07.30,"조회 41,374",댓글 18
8,댄디,훈훈 남친 룩,21.03.24,"조회 20,023",댓글 17
9,댄디,서머 데이트 룩,21.07.26,"조회 8,371",댓글 17


In [20]:
# csv 파일로 저장
df_.to_csv('codi.csv', encoding = 'utf-8-sig')